<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [33]:
import os
import pathlib
from glob import glob
from math import ceil, floor

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import rioxarray as rxr
import xarray as xr

In [21]:
matilija_path = os.path.join(
    pathlib.Path.home(),
    'Documents',
    'landslide-matilija',
    'data',
    'matilija-boundary',
    'boundary_latlon.zip'
)

os.path.exists(matilija_path)

True

In [24]:
gpd.read_file(matilija_path)

,FID,geometry
0,125199354,"POLYGON ((-119.44426 34.59883, -119.44337 34.5..."


In [8]:
chirps_path = os.path.join(
    pathlib.Path.home(),
    'My Drive', 
    'research/', 
    'landslide.regions.sync', 
    'regions_data', 
    'precip_chirps_monthly',
    'chirps-v2.0.monthly.nc'
)

os.path.exists(chirps_path)

True

In [26]:
chirps = xr.open_dataset(chirps_path)
chirps

<xarray.Dataset>
Dimensions:    (latitude: 2000, longitude: 7200, time: 479)
Coordinates:
  * latitude   (latitude) float32 -49.97 -49.92 -49.88 ... 49.87 49.92 49.97
  * longitude  (longitude) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time       (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2020-11-01
Data variables:
    precip     (time, latitude, longitude) float32 ...
Attributes: (12/15)
    Conventions:       CF-1.6
    title:             CHIRPS Version 2.0
    history:           created by Climate Hazards Group
    version:           Version 2.0
    date_created:      2020-12-16
    creator_name:      Pete Peterson
    ...                ...
    reference:         Funk, C.C., Peterson, P.J., Landsfeld, M.F., Pedreros,...
    comments:           time variable denotes the first day of the given month.
    acknowledgements:  The Climate Hazards Group InfraRed Precipitation with ...
    ftp_url:           ftp://chg-ftpout.geog.ucsb.edu/pub/org/chg/products/CH...
    website:           http://chg.geog.ucsb.edu/data/chirps/index.html
    faq:               http://chg-wiki.geog.ucsb.edu/wiki/CHIRPS_FAQ

In [18]:
%%time
chirps = xr.open_dataset(chirps_path)
chirps.mean()

CPU times: user 2min 13s, sys: 3min 32s, total: 5min 45s
Wall time: 7min 56s


<xarray.Dataset>
Dimensions:  ()
Data variables:
    precip   float32 72.36

In [30]:
chirps = xr.open_dataset(chirps_path, chunks={'time': 50})
chirps.chunks
chirps

<xarray.Dataset>
Dimensions:    (latitude: 2000, longitude: 7200, time: 479)
Coordinates:
  * latitude   (latitude) float32 -49.97 -49.92 -49.88 ... 49.87 49.92 49.97
  * longitude  (longitude) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time       (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2020-11-01
Data variables:
    precip     (time, latitude, longitude) float32 dask.array<chunksize=(50, 2000, 7200), meta=np.ndarray>
Attributes: (12/15)
    Conventions:       CF-1.6
    title:             CHIRPS Version 2.0
    history:           created by Climate Hazards Group
    version:           Version 2.0
    date_created:      2020-12-16
    creator_name:      Pete Peterson
    ...                ...
    reference:         Funk, C.C., Peterson, P.J., Landsfeld, M.F., Pedreros,...
    comments:           time variable denotes the first day of the given month.
    acknowledgements:  The Climate Hazards Group InfraRed Precipitation with ...
    ftp_url:           ftp://chg-ftpout.geog.ucsb.edu/pub/org/chg/products/CH...
    website:           http://chg.geog.ucsb.edu/data/chirps/index.html
    faq:               http://chg-wiki.geog.ucsb.edu/wiki/CHIRPS_FAQ

In [19]:
%%time
chirps = xr.open_dataset(chirps_path, chunks="auto")
chirps.mean()['precip'].values

CPU times: user 2min 34s, sys: 36 s, total: 3min 10s
Wall time: 2min 33s


array(72.36323, dtype=float32)

In [31]:
from dask.distributed import Client

# Setup a local cluster.
# By default this sets up 1 worker per core
client = Client()
client.cluster

In [32]:
%%time
chirps = xr.open_dataset(chirps_path, chunks="auto")
chirps.mean()['precip'].values

CPU times: user 4.89 s, sys: 2.6 s, total: 7.5 s
Wall time: 1min


array(72.36323, dtype=float32)

In [36]:
NDVI_PATH_PATTERN = os.path.join("data", "modis-lai", "*.hdf")
NDVI_CRS = (
    "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m "
    "+no_defs")
BOUNDARY_PATH = os.path.join("data", "matilija-boundary", "boundary.shp")
OUTPUT_PATH = "matilija-lai-mean.csv"

os.chdir(os.path.join(
    pathlib.Path.home(), "Documents", "landslide-matilija"))
os.path.exists(BOUNDARY_PATH)

True

In [41]:
ndvi_paths = glob(NDVI_PATH_PATTERN)
ndvi_paths.sort()
ndvi_paths[:5]

['data/modis-lai/MCD15A2H.A2004273.h08v05.006.2015094223135.hdf',
 'data/modis-lai/MCD15A2H.A2004281.h08v05.006.2015095090106.hdf',
 'data/modis-lai/MCD15A2H.A2004289.h08v05.006.2015095084023.hdf',
 'data/modis-lai/MCD15A2H.A2004297.h08v05.006.2015095081406.hdf',
 'data/modis-lai/MCD15A2H.A2004305.h08v05.006.2015095225749.hdf']

In [51]:
# Get dataset names
ds_names = []
for path in ndvi_paths:
    with rasterio.open(path) as raster:
        ds_names.append(raster.subdatasets[1])

/Users/elsa/opt/miniconda3/envs/ea-nbg/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [52]:
rxr.open_rasterio(ds_names[0], masked=True).squeeze() 

<xarray.DataArray (y: 2400, x: 2400)>
[5760000 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 -1.112e+07 -1.112e+07 ... -1.001e+07 -1.001e+07
  * y            (y) float64 4.448e+06 4.447e+06 ... 3.337e+06 3.336e+06
    spatial_ref  int64 0
Attributes:
    scale_factor:  0.1
    add_offset:    0.0
    long_name:     MOD15A2H MODIS/Terra Gridded 500M Leaf Area Index LAI (8-d...
    units:         m^2/m^2

In [53]:
# Open NDVI files with xarray
ndvi_xr = xr.concat(
    [rxr.open_rasterio(ds, masked=True).squeeze() 
    for ds in ds_names], 
    dim='time')
ndvi_xr

<xarray.DataArray (time: 770, y: 2400, x: 2400)>
array([[[254., 254., 254., ...,   2.,   2.,   2.],
        [254., 254., 254., ...,   2.,   2.,   2.],
        [254., 254., 254., ...,   3.,   3.,   2.],
        ...,
        [  2.,   3.,   2., ...,   5.,   5.,   5.],
        [  2.,   3.,   3., ...,   5.,   5.,   5.],
        [  2.,   2.,   2., ...,   5.,   5.,   6.]],

       [[254., 254., 254., ...,   2.,   2.,   2.],
        [254., 254., 254., ...,   2.,   2.,   2.],
        [254., 254., 254., ...,   2.,   3.,   3.],
        ...,
        [  1.,   2.,   2., ...,   5.,   5.,   5.],
        [  2.,   2.,   2., ...,   5.,   5.,   5.],
        [  2.,   2.,   2., ...,   5.,   7.,   5.]],

       [[254., 254., 254., ...,   1.,   1.,   1.],
        [254., 254., 254., ...,   2.,   2.,   2.],
        [254., 254., 254., ...,   2.,   2.,   2.],
        ...,
...
        ...,
        [  3.,   2.,   3., ...,   4.,   4.,   3.],
        [  2.,   2.,   3., ...,   4.,   4.,   3.],
        [  2.,   3.,   2., ...,   3.,   3.,   3.]],

       [[254., 254., 254., ...,   2.,   2.,   3.],
        [254., 254., 254., ...,   2.,   2.,   2.],
        [254., 254., 254., ...,   2.,   2.,   2.],
        ...,
        [  3.,   3.,   2., ...,   3.,   3.,   3.],
        [  3.,   2.,   3., ...,   4.,   3.,   3.],
        [  3.,   2.,   3., ...,   3.,   3.,   3.]],

       [[254., 254., 254., ...,   2.,   2.,   2.],
        [254., 254., 254., ...,   2.,   2.,   3.],
        [254., 254., 254., ...,   2.,   2.,   2.],
        ...,
        [  2.,   2.,   3., ...,   5.,   4.,   3.],
        [  3.,   3.,   3., ...,   4.,   4.,   4.],
        [  2.,   3.,   3., ...,   3.,   4.,   4.]]], dtype=float32)
Coordinates:
    band         int64 1
  * x            (x) float64 -1.112e+07 -1.112e+07 ... -1.001e+07 -1.001e+07
  * y            (y) float64 4.448e+06 4.447e+06 ... 3.337e+06 3.336e+06
    spatial_ref  int64 0
Dimensions without coordinates: time
Attributes:
    scale_factor:  0.1
    add_offset:    0.0
    long_name:     MOD15A2H MODIS/Terra Gridded 500M Leaf Area Index LAI (8-d...
    units:         m^2/m^2